## Escuela Superior de Cómputo del IPN.
### Matematicas Avanzadas para la Ciencia de Datos
Profesor: Alfonso
<br> 
Alumno: Huerta Ramírez  Michael Adolfo             

<center>Segundo parcial - 14 de Diciembre de 2022 </center>
<br>
Boleta: 2020602866

1. Considere el siguiente problema de programación lineal

a) Construya el problema dual asociado

$ 2y_1 + 3y_2  $
<br>
$y_1 + 2y_2 <= 2$
<br>
$2y_1 - y_2 <= 3$
<br>
$3y_1 + y_2 <= 5$
<br>
$y_1 - 3y_2 <= 6$



b) Resuelva el problema anterior geométricamente

c) Utilice informaci ́on acerca del poblema lineal dual y los teoremas de
dualidad para resolver el problema primal

2. Resuelva el problema anterior utilizando una implementaci ́on propia del
método dual simplex


3. Considere el siguiente programa lineal

    a) Escriba el problema dual y verifique que (4, 5) es una solucin factible.
    
    b) Use la información de la parte a) para derivar una solución  ́optima para los problemas dual y primal.